<a href="https://colab.research.google.com/github/dhruvchopra2003/FeelTunes/blob/main/EmotionDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!nvidia-smi

Mon Apr 10 19:09:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!unzip '/content/gdrive/MyDrive/Project_1/archive(5).zip' -d /content

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
# initialize data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Preprocessing all test data
train_generator = train_data_gen.flow_from_directory(
    '/content/train',
    target_size = (48,48),
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = 'categorical')

Found 28709 images belonging to 7 classes.


In [ ]:
# Preprocess all training images
validation_generator = validation_data_gen.flow_from_directory(
    '/content/test',
    target_size = (48,48), # setting input shape to 48,48
    batch_size = 64, 
    color_mode = "grayscale", # setting color as 1 (grayscale)
    class_mode = 'categorical')

Found 7178 images belonging to 7 classes.


In [ ]:
# Create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3,3), activation= 'relu', input_shape=(48,48,1))) #input_shape: 48,48 & grayscale
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25)) # to avoid overfitting

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25)) #to avoid overfitting

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
# in the final dense layer, the 7 outputs are the 7 categories, outputted with softmax

In [ ]:
emotion_model.compile(loss = 'categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])

In [ ]:
emotion_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 128)      0         
 2D)                                                    

In [ ]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/50


<ipython-input-15-ae419f95bfec>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


448/448 [==============================] - 29s 47ms/step - loss: 1.8018 - accuracy: 0.2610 - val_loss: 1.7251 - val_accuracy: 0.3149
Epoch 2/50
448/448 [==============================] - 20s 45ms/step - loss: 1.6405 - accuracy: 0.3579 - val_loss: 1.5548 - val_accuracy: 0.4134
Epoch 3/50
448/448 [==============================] - 17s 39ms/step - loss: 1.5249 - accuracy: 0.4135 - val_loss: 1.4525 - val_accuracy: 0.4487
Epoch 4/50
448/448 [==============================] - 20s 45ms/step - loss: 1.4429 - accuracy: 0.4489 - val_loss: 1.4119 - val_accuracy: 0.4616
Epoch 5/50
448/448 [==============================] - 17s 39ms/step - loss: 1.3793 - accuracy: 0.4749 - val_loss: 1.3318 - val_accuracy: 0.4954
Epoch 6/50
448/448 [==============================] - 17s 39ms/step - loss: 1.3234 - accuracy: 0.4974 - val_loss: 1.3154 - val_accuracy: 0.4987
Epoch 7/50
448/448 [==============================] - 20s 45ms/step - loss: 1.2866 - accuracy: 0.5113 - val_loss: 1.2684 - val_accuracy: 0.5159
Epo

In [ ]:
# saving the model
model_json = emotion_model.to_json()

with open("emotion_model.json","w") as json_file:
  json_file.write(model_json)

In [ ]:
# save trained model weights in .h5 file

emotion_model.save_weights('emotion_model.h5')